**Las funciones que se nos solicitan son:**

    ► Cantidad de películas estrenadas en un mes

    ► Cantidad de filmaciones estrenadas en un día

    ► Título, año de estreno y score de la película ingresada

    ► Título, cantidad de votos y valor promedio de las votaciones. La misma deberá contar con al menos 2000 valoraciones

    ► Nombre de actor, éxito del mismo medido a través del retorno, cantidad de películas en las que participó y promedio de retorno
    
    ► Nombre de director, éxito del mismo medido a través del retorno, nombre de cada película en la que participó con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [22]:
import pandas as pd
import numpy as np
import unicodedata
from datetime import datetime
import ast
import re

In [23]:
df = pd.read_csv("Movies_EDA.csv", sep=",")

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               42195 non-null  int64  
 1   title            42195 non-null  object 
 2   overview         42195 non-null  object 
 3   release_date     42195 non-null  object 
 4   release_year     42195 non-null  int64  
 5   budget           42195 non-null  float64
 6   return           42195 non-null  float64
 7   popularity       42195 non-null  float64
 8   vote_average     42195 non-null  float64
 9   vote_count       42195 non-null  float64
 10  revenue          42195 non-null  float64
 11  runtime          42195 non-null  float64
 12  genres_names     42195 non-null  object 
 13  names_actors     42195 non-null  object 
 14  director         42195 non-null  object 
 15  companies_names  42195 non-null  object 
dtypes: float64(7), int64(2), object(7)
memory usage: 5.2+ MB


Eliminamos duplicados de la columna id.

In [25]:
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['id'],inplace=True)
df=df.reset_index(drop=True)

In [26]:
df.drop(['id', 'overview', 'runtime', 'genres_names', 'companies_names'], axis=1, inplace=True)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         42195 non-null  object 
 1   release_date  42195 non-null  object 
 2   release_year  42195 non-null  int64  
 3   budget        42195 non-null  float64
 4   return        42195 non-null  float64
 5   popularity    42195 non-null  float64
 6   vote_average  42195 non-null  float64
 7   vote_count    42195 non-null  float64
 8   revenue       42195 non-null  float64
 9   names_actors  42195 non-null  object 
 10  director      42195 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 3.5+ MB


API 1.

In [28]:
#Cambiamos el tipo de dato de release_date a cual corresponda
df['release_date'] = pd.to_datetime(df['release_date'])

def cantidad_filmaciones_mes(mes):
    # Filtramos el DataFrame por el mes especificado
    df_mes = df[df['release_date'].dt.month_name(locale='es').str.lower() == mes.lower()]

    # Obtenemos la cantidad de películas en el mes consultado
    cantidad = len(df_mes)

    # Retornar el resultado en un diccionario
    return f"Cantidad: {cantidad}\nMes: {mes.capitalize()}"


In [29]:
resultado = cantidad_filmaciones_mes("julio")
print(resultado)

Cantidad: 2464
Mes: Julio


API 2.

In [30]:
def remover_acentos(texto):
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    texto_sin_acentos = ''.join(c for c in texto_normalizado if not unicodedata.combining(c))
    return texto_sin_acentos

def cantidad_filmaciones_dia(dia):
    # Normalizamos el día ingresado para tener en cuenta los acentos
    dia_normalizado = remover_acentos(dia)

    # Filtramos el DataFrame por el día especificado
    df_dia = df[df['release_date'].dt.day_name(locale='es').str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') == dia_normalizado.lower()]

    # Obtenemos la cantidad de películas en el día consultado
    cantidad = len(df_dia)

    return f"Día: {dia.capitalize()}\nCantidad de filmaciones: {cantidad}"

In [31]:
cantidad = cantidad_filmaciones_dia("martes")
print(cantidad)

Día: Martes
Cantidad de filmaciones: 4333


API 3.

In [32]:
def score_titulo(titulo_de_la_filmacion):
    # Convertimos el título de la filmación a minúsculas
    titulo_busqueda = titulo_de_la_filmacion.lower()

    # Filtramos el DataFrame por el título de la filmación
    df_filmacion = df[df['title'].str.lower() == titulo_busqueda]

    # Verificamos si se encontró la filmación
    if len(df_filmacion) == 0:
        return "No se encontró la filmación especificada."

    # Obtenemos los datos de interés
    titulo = df_filmacion['title'].values[0]
    año_estreno = df_filmacion['release_year'].values[0]
    score = round(df_filmacion['popularity'].values[0], 2)

    return f"Película: {titulo}\nAño de estreno: {año_estreno}\nPuntuación: {score}"

In [33]:
score = score_titulo("A Perfect World")
print(score)

Película: A Perfect World
Año de estreno: 1993
Puntuación: 8.04


API 4.

In [34]:
def votos_titulo(titulo_de_la_filmacion):
    # Filtramos el DataFrame por el título de la filmación
    df_filmacion = df[df['title'] == titulo_de_la_filmacion]

    # Verificamos si se encontró la filmación
    if len(df_filmacion) == 0:
        return "No se encontró la filmación especificada."

    # Obtenemos los datos de interés
    titulo = df_filmacion['title'].values[0]
    cantidad_votos = int(df_filmacion['vote_count'].values[0])
    promedio_votos = round(df_filmacion['vote_average'].values[0], 2)
    año_estreno = df_filmacion['release_year'].values[0]

    # Verificamos si cumple con la condición de al menos 2000 valoraciones
    if cantidad_votos < 2000:
        return "La película no cumple con la condición de tener al menos 2000 valoraciones."

    return f"Película: {titulo}\nAño de estreno: {año_estreno}\nCantidad de votos: {cantidad_votos}\nPromedio de valoraciones: {promedio_votos}"

In [35]:
titulo = "Jumanji"
resultado = votos_titulo(titulo)
print(resultado)

Película: Jumanji
Año de estreno: 1995
Cantidad de votos: 2413
Promedio de valoraciones: 6.9


API 5.

In [36]:
def get_actor(nombre_actor):
    cantidad_peliculas = 0
    total_retorno = 0

    for index, row in df.iterrows():
        nombres_actores = row['names_actors']
        retorno = row['return']

        if nombre_actor.lower() in nombres_actores.lower():
            cantidad_peliculas += 1
            total_retorno += retorno

    if cantidad_peliculas == 0:
        return "No se encontró al actor especificado."

    promedio_retorno = round((total_retorno / cantidad_peliculas), 2)

    return f"Actor: {nombre_actor}\nParticipación en películas: {cantidad_peliculas}\nPromedio de retorno: {promedio_retorno}"


In [37]:
nombre_actor = "Tom Hanks" 
resultado = get_actor(nombre_actor)
print(resultado)

Actor: Tom Hanks
Participación en películas: 67
Promedio de retorno: 2.61


API 6.

In [38]:
def get_director(nombre_director):
    resultado = ""

    for index, row in df.iterrows():
        director = row['director']
        if isinstance(director, str) and nombre_director.lower() == director.lower():
            titulo = row['title']
            fecha_lanzamiento = pd.to_datetime(row['release_date']).date()
            retorno = round(row['return'], 2)
            costo = int(row['budget'])
            ganancia = int(row['revenue'])
            
            resultado += f"Película: {titulo}\nFecha de lanzamiento: {fecha_lanzamiento}\nRetorno: {retorno}\nCosto: {costo}\nGanancia: {ganancia}\n\n"
    
    if resultado == "":
        resultado = "No se encontró al director especificado."

    return resultado

In [39]:
nombre_director = "James Cameron"
resultado = get_director(nombre_director)
print(resultado)

Película: True Lies
Fecha de lanzamiento: 1994-07-14
Retorno: 3.29
Costo: 115000000
Ganancia: 378882411

Película: Terminator 2: Judgment Day
Fecha de lanzamiento: 1991-07-01
Retorno: 5.2
Costo: 100000000
Ganancia: 520000000

Película: The Abyss
Fecha de lanzamiento: 1989-08-09
Retorno: 1.29
Costo: 70000000
Ganancia: 90000098

Película: Aliens
Fecha de lanzamiento: 1986-07-18
Retorno: 9.91
Costo: 18500000
Ganancia: 183316455

Película: The Terminator
Fecha de lanzamiento: 1984-10-26
Retorno: 12.25
Costo: 6400000
Ganancia: 78371200

Película: Titanic
Fecha de lanzamiento: 1997-11-18
Retorno: 9.23
Costo: 200000000
Ganancia: 1845034188

Película: Piranha Part Two: The Spawning
Fecha de lanzamiento: 1981-01-01
Retorno: 0.0
Costo: 145786
Ganancia: 0

Película: Ghosts of the Abyss
Fecha de lanzamiento: 2003-04-10
Retorno: 1.31
Costo: 13000000
Ganancia: 17040871

Película: Aliens of the Deep
Fecha de lanzamiento: 2005-01-21
Retorno: 0.0
Costo: 0
Ganancia: 0

Película: Avatar
Fecha de lanzamie

Tanto la API 5 como la API 6 están normalizadas para que se puedan leer sin discriminar entre mayúsculas y minúsculas.